# Reference data accuracy assessment by Radiant Earth

Radiant Earth is conducting an accuracy assessment of DE Africa cropmask reference data using the airbus high-res satellite archive. This notebook produces a confusion matrix between DE AFrica's labels and Radiant Earth's labels.  

Inputs will be:

1. `<AEZ-region_RE_sample_validation.geojson>` : The results from collecting training data in the CEO tool

Output will be:
1. A `confusion error matrix` containing Overall, Producer's, and User's accuracy, along with the F1 score.

***

In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

/env/lib/python3.6/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.2-CAPI-1.11.0 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Analysis Parameters

In [ ]:
folder = 'data/training_validation/collect_earth/southern/'
gjson =  'data/training_validation/collect_earth/southern/Southern_region_RE_sample_validated.geojson'

## Run this if doing validation results for the entire continent

In [2]:
so='data/training_validation/collect_earth/southern/Southern_region_RE_sample_validated.geojson'
sa='data/training_validation/collect_earth/sahel/Sahel_region_RE_sample_validated.geojson'
w='data/training_validation/collect_earth/western/Western_region_RE_sample_validated.geojson'
e='data/training_validation/collect_earth/eastern/Eastern_region_RE_sample_validated.geojson'
n='data/training_validation/collect_earth/northern/Northern_region_RE_sample_validated.geojson'
io='data/training_validation/collect_earth/indian_ocean/Indian_ocean_region_RE_sample_validated.geojson'
c='data/training_validation/collect_earth/central/Central_region_RE_sample_validated.geojson'

so=gpd.read_file(so)
sa=gpd.read_file(sa)
w=gpd.read_file(w)
e=gpd.read_file(e)
n=gpd.read_file(n)
io=gpd.read_file(io)
c=gpd.read_file(c)

df = pd.concat([so,sa,w,e,n,io,c]).drop(columns=['smpl_class', 'SMPL_SAMPLEID', 'smpl_gfsad_samp','smpl_sampleid']).reset_index(drop=True)
df.head()

,Class,Validation_Class,geometry
0,crop,crop,"POLYGON ((30.13818 -17.49899, 30.13860 -17.498..."
1,non-crop,non-crop,"POLYGON ((32.84175 -16.64361, 32.84216 -16.643..."
2,crop,crop,"POLYGON ((32.31192 -28.46689, 32.31234 -28.466..."
3,non-crop,non-crop,"POLYGON ((19.86882 -25.14779, 19.86923 -25.147..."
4,non-crop,non-crop,"POLYGON ((19.02961 -31.68897, 19.03003 -31.688..."


## Otherwise, run this cell

In [ ]:
#ground truth shapefile
df = gpd.read_file(gjson)
df.head()

### Clean up dataframe


In [3]:
#rename columns
df = df.rename(columns={'Class':'Prediction',
                        'Validation_Class':'Actual'})
df.head()

,Prediction,Actual,geometry
0,crop,crop,"POLYGON ((30.13818 -17.49899, 30.13860 -17.498..."
1,non-crop,non-crop,"POLYGON ((32.84175 -16.64361, 32.84216 -16.643..."
2,crop,crop,"POLYGON ((32.31192 -28.46689, 32.31234 -28.466..."
3,non-crop,non-crop,"POLYGON ((19.86882 -25.14779, 19.86923 -25.147..."
4,non-crop,non-crop,"POLYGON ((19.02961 -31.68897, 19.03003 -31.688..."


***

### Reclassify prediction & actual columns

1 = crop, 
0 = non-crop

In [4]:
df['Prediction'] = np.where(df['Prediction']=='non-crop', 0, df['Prediction'])
df['Prediction'] = np.where(df['Prediction']=='crop', 1, df['Prediction'])

df['Actual'] = np.where(df['Actual']=='non-crop', 0, df['Actual'])
df['Actual'] = np.where(df['Actual']=='crop', 1, df['Actual'])

df.head()

,Prediction,Actual,geometry
0,1,1,"POLYGON ((30.13818 -17.49899, 30.13860 -17.498..."
1,0,0,"POLYGON ((32.84175 -16.64361, 32.84216 -16.643..."
2,1,1,"POLYGON ((32.31192 -28.46689, 32.31234 -28.466..."
3,0,0,"POLYGON ((19.86882 -25.14779, 19.86923 -25.147..."
4,0,0,"POLYGON ((19.02961 -31.68897, 19.03003 -31.688..."


### Generate a confusion matrix with all classes

In [5]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,289,18,307
1,3,258,261
N/A,10,2,12
mixed,8,32,40
All,310,310,620


### Reclassify into a binary assessment

In [6]:
counts = df.groupby('Actual').count()

print("Total number of samples: " + str(len(df)))
print("Number of 'mixed' samples: "+ str(counts[counts.index=='mixed']['Prediction'].values[0]))
print("Number of 'N/A' samples: "+ str(counts[counts.index=='N/A']['Prediction'].values[0]))

print("Dropping 'mixed' and 'N/A' samples")

df = df.drop(df[df['Actual']=='mixed'].index)
df = df.drop(df[df['Actual']=='N/A'].index)

Total number of samples: 620
Number of 'mixed' samples: 40
Number of 'N/A' samples: 12
Dropping 'mixed' and 'N/A' samples


---

### Recreate confusion matrix

In [7]:
confusion_matrix = pd.crosstab(df['Actual'],
                               df['Prediction'],
                               rownames=['Actual'],
                               colnames=['Prediction'],
                               margins=True)

confusion_matrix

Prediction,0,1,All
Actual,,,
0,289,18,307
1,3,258,261
All,292,276,568


In [ ]:
# confusion_matrix.to_csv('radiant_earth_reference_data_accuracy_continental_results.csv')

### Calculate User's and Producer's Accuracy

`Producer's Accuracy`

In [8]:
confusion_matrix["Producer's"] = [confusion_matrix.loc[0, 0] / confusion_matrix.loc[0, 'All'] * 100,
                              confusion_matrix.loc[1, 1] / confusion_matrix.loc[1, 'All'] * 100,
                              np.nan]

`User's Accuracy`

In [9]:
users_accuracy = pd.Series([confusion_matrix[0][0] / confusion_matrix[0]['All'] * 100,
                                confusion_matrix[1][1] / confusion_matrix[1]['All'] * 100]
                         ).rename("User's")

confusion_matrix = confusion_matrix.append(users_accuracy)

`Overall Accuracy`

In [10]:
confusion_matrix.loc["User's","Producer's"] = (confusion_matrix.loc[0, 0] + 
                                                confusion_matrix.loc[1, 1]) / confusion_matrix.loc['All', 'All'] * 100

`F1 Score`

The F1 score is the harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 (perfect precision and recall), and is calculated as:

$$
\begin{aligned}
\text{Fscore} = 2 \times \frac{\text{UA} \times \text{PA}}{\text{UA} + \text{PA}}.
\end{aligned}
$$

Where UA = Users Accuracy, and PA = Producer's Accuracy

In [11]:
fscore = pd.Series([(2*(confusion_matrix.loc["User's", 0]*confusion_matrix.loc[0, "Producer's"]) / (confusion_matrix.loc["User's", 0]+confusion_matrix.loc[0, "Producer's"])) / 100,
                    f1_score(df['Actual'].astype(np.int8), df['Prediction'].astype(np.int8), average='binary')]
                         ).rename("F-score")

confusion_matrix = confusion_matrix.append(fscore)

### Tidy Confusion Matrix

* Limit decimal places,
* Add readable class names
* Remove non-sensical values 

In [12]:
# round numbers
confusion_matrix = confusion_matrix.round(decimals=2)

In [13]:
# rename booleans to class names
confusion_matrix = confusion_matrix.rename(columns={0:'Non-crop', 1:'Crop', 'All':'Total'},
                                            index={0:'Non-crop', 1:'Crop', 'All':'Total'})

In [14]:
#remove the nonsensical values in the table
confusion_matrix.loc["User's", 'Total'] = '--'
confusion_matrix.loc['Total', "Producer's"] = '--'
confusion_matrix.loc["F-score", 'Total'] = '--'
confusion_matrix.loc["F-score", "Producer's"] = '--'

In [15]:
confusion_matrix

Prediction,Non-crop,Crop,Total,Producer's
Actual,,,,
Non-crop,289.00,18.00,307,94.14
Crop,3.00,258.00,261,98.85
Total,292.00,276.00,568,--
User's,98.97,93.48,--,96.3
F-score,0.96,0.96,--,--


### Export csv

In [16]:
confusion_matrix.to_csv(folder+ 'radiant_earth_reference_data_accuracy_results.csv')